In [1]:
import pandas as pd
#按batch_size分
from torch.utils.data import DataLoader,TensorDataset,Dataset
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
batch_size=16

In [2]:
# 定义日志（data文件夹下，同级目录新建一个data文件夹）
import time
import datetime
import pytz
tz = pytz.timezone('Asia/Shanghai')
def write_log(w):
    file_name = '../data/' + datetime.date.today().strftime('%m%d') + "_{}.log".format("bert_base4")
    t0 = datetime.datetime.now(tz).strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info + '\n')
# write_log('test')

In [3]:
# added_token=['##char##']
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese",additional_special_tokens=added_token)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def text2token(text,tokenizer,max_length=100):
    text2id = tokenizer(
        text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
    )
    input_ids=text2id["input_ids"].tolist()
    attention_mask=text2id["attention_mask"].tolist()
    return input_ids,attention_mask
def data2token(data_,tokenizer):
    text=[i for i in data_['title'].values]
    input_ids,attention_mask=text2token(text,tokenizer)
    data_['input_ids']=input_ids
    data_['attention_mask']=attention_mask
    return data_

In [4]:
class SentimentDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "title"]
        label = self.dataset.loc[idx, "label"]
        pre = self.dataset.loc[idx, "pre"]
        input_ids = self.dataset.loc[idx, "input_ids"]
        attention_mask = self.dataset.loc[idx, "attention_mask"]
        sample = {"text": text, "label": label,"pre":pre,"input_ids":input_ids,"attention_mask":attention_mask}
        # print(sample)
        return sample
    
bad_case=pd.read_csv("../data/data_test1_bad.csv")
bad_case=data2token(bad_case,tokenizer)
bad_loader = DataLoader(
    SentimentDataset(bad_case), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0
)

In [5]:
class SentimentDataset(Dataset):
    def __init__(self,df):
        self.dataset = df
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        text = self.dataset.loc[idx, "title"]
        label = self.dataset.loc[idx, "label"]
        input_ids = self.dataset.loc[idx, "input_ids"]
        attention_mask = self.dataset.loc[idx, "attention_mask"]
        sample = {"text": text, "label": label,"input_ids":input_ids,"attention_mask":attention_mask}
        # print(sample)
        return sample


data_test1=pd.read_csv("../data/data_test1.csv")
data_test1=data2token(data_test1,tokenizer)
test1_loader = DataLoader(
    SentimentDataset(data_test1), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

data_test2=pd.read_csv("../data/data_test2.csv")
data_test2=data2token(data_test2,tokenizer)
test2_loader = DataLoader(
    SentimentDataset(data_test2), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

data_train=pd.read_csv("../data/data_train.csv")
data_train=data2token(data_train,tokenizer)
batch_size=16
train_loader = DataLoader(
    SentimentDataset(data_train), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0
)

data_val=pd.read_csv("../data/data_val.csv")
data_val=data2token(data_val,tokenizer)
test_loader = DataLoader(
    SentimentDataset(data_val), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

In [6]:
from tqdm import tqdm
from sklearn import metrics
def predict_loader(device,test_loader,cls):
    with torch.no_grad():
        cls.to(device)
        cls.eval()
        output_all=[]
        label_all=[]
        for batch_idx,batch in enumerate(test_loader):
            print(str(batch_idx)+'/'+str(len(test_loader)),end='\r')
            label=batch['label'].to(device)#batch size * 1
            label_all.append(label.view(-1,1))
            input_ids=torch.stack(batch['input_ids']).t().to(device)#batch size * 100
            attention_mask=torch.stack(batch['attention_mask']).t().to(device)#batch size * 100
            
            #计算输出
            output = cls(input_ids, attention_mask=attention_mask)#batch size * 1
            
            #四舍五入
            softmax = nn.Softmax(dim=1)
            output=softmax(output)
            output=output.argmax(dim=1)
            output_all.append(output)
        output_all=torch.cat(output_all,0)
        label_all=torch.cat(label_all,0)

        output_all=np.array(output_all.cpu())
        label_all=np.array(label_all.cpu())
        acc_score=metrics.accuracy_score(label_all,output_all)
        print("准确率:"+str(acc_score))
        print(metrics.classification_report(label_all,output_all))
        return label_all,output_all
        

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F

class fn_cls(nn.Module):
    def __init__(self,device):
        super(fn_cls, self).__init__()
        self.model = AutoModel.from_pretrained("bert-base-uncased")
        self.model.resize_token_embeddings(len(tokenizer))##############
        self.model.to(device)
#         self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(768, 4)

    def forward(self, x, attention_mask=None):
        outputs = self.model(x, attention_mask=attention_mask)
#         print(outputs[0])torch.Size([8, 100, 768])
#         print(outputs[1])torch.Size([8, 768])
#         print(outputs[0][:,0,:])torch.Size([8, 768])
        x = outputs[1]
#         x = self.dropout(x)
        x = self.l1(x)
        return x


import torch
device0 = torch.device('cuda:7' if torch.cuda.is_available() else "cpu")#训练集gpu
softmax = nn.Softmax(dim=1)
criterion = nn.CrossEntropyLoss()#weight=weight

In [15]:
# model1对每个badcase预测，然后修正model2
def train_one_bad_epoch(device_train,lmd=10):
    softmax = nn.Softmax(dim=1)
    cls2.to(device_train)
    
    epoch_loss=0
    total=0
    correct=0
    output_all=[]
    label_all=[]
    for batch_idx,batch in enumerate(bad_loader):
#         print('___________batch'+str(batch_idx)+'___________')
        label=batch['label'].to(device_train)#batch size * 1
        pre=batch['pre'].to(device_train)
        input_ids=torch.stack(batch['input_ids']).t().to(device_train)#batch size * 100
        attention_mask=torch.stack(batch['attention_mask']).t().to(device_train)#batch size * 100
                
        output2=cls2(input_ids, attention_mask=attention_mask)
        output2=softmax(output2)
        
        loss=0
        for i in range(len(output2)):
#             print(output[i].tolist(),'\n',output2[i].tolist(),'\n','\n')
            loss = loss - torch.log(1-output2[i][pre[i]])
            loss=loss/lmd
        
        optimizer2.zero_grad() # 将所有参数的梯度都置零
        loss.backward()    # 误差反向传播计算参数梯度
        optimizer2.step()    # 通过梯度做一步参数更新
        
        with torch.no_grad():
            total+=len(output2)
            epoch_loss+=loss.item()
            output2=output2.argmax(dim=1)
            add_correct=(output2== pre).sum().item()
            correct+=add_correct
            acc=correct/total
        print(str(batch_idx)+'/'+str(len(bad_loader))+'\tbatch_loss:'+str(loss.item())+'\tbad_acc:'+str(acc),end='\r')
        
        
    return epoch_loss,acc
        

In [9]:
from sklearn import metrics
import numpy as np
from tqdm import tqdm

def test(device_test):
    cls2.to(device_test)
    cls2.eval()

    epoch_loss=0
    total=0
    correct=0
    output_all=[]
    label_all=[]
    for batch_idx,batch in enumerate(test1_loader):
        with torch.no_grad():
#             print(batch['label'])
            label=batch['label'].to(device_test)#batch size * 1
            label_all.append(label.view(-1,1))
            input_ids=torch.stack(batch['input_ids']).t().to(device_test)#batch size * 100
            attention_mask=torch.stack(batch['attention_mask']).t().to(device_test)#batch size * 100
            
            #计算输出
            output = cls2(input_ids, attention_mask=attention_mask)#batch size * 1
            total+=len(output)
            
            #计算loss
            
#             print(output,label)
            loss = criterion(output, label)
            epoch_loss+=loss
            ave_loss=epoch_loss/total
            
            #四舍五入
            output=softmax(output)
            output=output.argmax(dim=1)
            output_all.append(output)
            
            #计算准确率
            add_correct=(output== label).sum().item()
            correct+=add_correct
            acc=correct/total
            
            if batch_idx%5==0:
                print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{}，ave_loss：{}'.format(
                    batch_idx, len(test1_loader),100.*batch_idx/len(test1_loader), 
                    correct, total,acc,
                    ave_loss
                    ),end= "\r")
            
            
            
    #结束：
    write_log('正确分类的样本数：{}，样本总数：{}，准确率：{}，ave_loss：{}'.format(
                    correct, total,acc,
                    ave_loss))
    
#     can't convert cuda:5 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
    output_all=torch.cat(output_all,0)
    label_all=torch.cat(label_all,0)
    
    output_all=np.array(output_all.cpu())
    label_all=np.array(label_all.cpu())
    acc_score=metrics.accuracy_score(label_all,output_all)
    print(metrics.classification_report(label_all,output_all))
    write_log("准确率:"+str(acc_score))
    
    return acc,epoch_loss.item()

# test(device1)

In [19]:
def train_one_epoch(min_test_epoch_loss,device_train,epoch_num,lmd=10):
    write_log("______________________________________________")
    write_log("______________________________________________")
    write_log("_______________train epoch"+str(epoch_num)+" start_______________")
    write_log("______________________________________________")
    write_log("______________________________________________")
    cls2.to(device_train)
    cls2.train()

    epoch_loss=0
    total=0
    correct=0
    acc=0
    ave_loss=0
    output_all=[]
    label_all=[]
    for batch_idx,batch in enumerate(train_loader):
        if batch_idx%500==0:
            write_log('正确分类的样本数：{}，样本总数：{}，准确率：{}，ave_loss：{}'.format(
                    correct, total,acc,
                    ave_loss))
            write_log("___train_one_bad_epoch___")
            epoch_loss,acc=train_one_bad_epoch(device0,lmd)
            write_log("bad_epoch_loss:"+str(epoch_loss)+"\tbad_acc:"+str(acc))
            
        label=batch['label'].to(device_train)#batch size * 1
        label_all.append(label.view(-1,1))
        input_ids=torch.stack(batch['input_ids']).t().to(device_train)#batch size * 100
        attention_mask=torch.stack(batch['attention_mask']).t().to(device_train)#batch size * 100

        #计算输出
        output = cls2(input_ids, attention_mask=attention_mask)#batch size * 1

        #计算loss
        loss = criterion(output, label)
        loss.backward()
        optimizer2.step()
        optimizer2.zero_grad()
        
        
        with torch.no_grad():
            #四舍五入
            output=softmax(output)
            output=output.argmax(dim=1)
            output_all.append(output)
            total+=len(output)
            
            #epoch_loss
            epoch_loss+=loss
            ave_loss=epoch_loss/total
            
            #计算准确率
            add_correct=(output== label).sum().item()
            correct+=add_correct
            acc=correct/total
            
            if batch_idx%5==0:
                print('[{}/{} ({:.0f}%)]\t正确分类的样本数：{}，样本总数：{}，准确率：{}，ave_loss：{}'.format(
                    batch_idx, len(train_loader),100.*batch_idx/len(train_loader), 
                    correct, total,acc,
                    ave_loss
                    ),end= "\r")
            
            
            
    #结束：
    
#     can't convert cuda:5 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
    with torch.no_grad():
        output_all=torch.cat(output_all,0)
        label_all=torch.cat(label_all,0)

        output_all=np.array(output_all.cpu())
        label_all=np.array(label_all.cpu())
        acc_score=metrics.accuracy_score(label_all,output_all)
        
#     print(metrics.classification_report(label_all,output_all))
    write_log('__________________train end__________________')
    write_log('正确分类的样本数：{}，样本总数：{}，准确率：{}，ave_loss：{}'.format(
                    correct, total,acc,
                    ave_loss))
    
    write_log("准确率:"+str(acc_score))
    
    write_log('__________________test start__________________')
    test_acc,test_epoch_loss=test(device0)
    if min_test_epoch_loss>test_epoch_loss:
        min_test_epoch_loss=test_epoch_loss
        write_log("store model")
        end = time.time()
        torch.save(cls2,"../data/cls_bad_"+str(epoch_num)+'_'+str(round(test_acc,5))+'_'+str(round(test_epoch_loss,5))+".model")
    
    write_log('__________________test end__________________')
    write_log('train_acc:'+str(acc)+'  train_epoch_loss:'+str(epoch_loss.item())+'  test_acc:'+str(test_acc)+'  test_epoch_loss:'+str(test_epoch_loss))
    
    
    train_acc_l.append(acc)
    train_epoch_loss_l.append(epoch_loss.item())
    test_acc_l.append(test_acc)
    test_epoch_loss_l.append(test_epoch_loss)
    write_log("______________________________________________")
    write_log("______________________________________________")
    write_log("_______________train epoch "+str(epoch_num)+" end_______________")
    write_log("______________________________________________")
    write_log("______________________________________________")
    return min_test_epoch_loss
    

    
def train(epoch_num):
    min_test_epoch_loss=999999
    for i in range(epoch_num):
        min_test_epoch_loss=train_one_epoch(min_test_epoch_loss,device0,i)

    
# train_one_epoch(device0,0)
# train()

In [13]:
cls=torch.load("../data/cls_6_0.88785_266.58456.model",map_location=device0)
cls2=torch.load("../data/cls_6_0.88785_266.58456.model",map_location=device0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from torch import optim
# cls2.load_state_dict(cls.state_dict())
optimizer2 = optim.Adam(cls2.parameters(), lr=1e-6)

train_acc_l=[]
train_epoch_loss_l=[]
test_acc_l=[]
test_epoch_loss_l=[]
train(10)


21:26:09 : ______________________________________________
21:26:09 : ______________________________________________
21:26:09 : _______________train epoch0 start_______________
21:26:09 : ______________________________________________
21:26:09 : ______________________________________________
21:26:09 : 正确分类的样本数：0，样本总数：0，准确率：0，ave_loss：0
21:26:09 : ___train_one_bad_epoch___
21:26:31 : bad_epoch_loss:27.687273893505335	bad_acc:0.7703885194259713
21:27:31 : 正确分类的样本数：7300，样本总数：8000，准确率：0.9125，ave_loss：0.02005632035434246.020037708804011345
21:27:31 : ___train_one_bad_epoch___
21:27:53 : bad_epoch_loss:27.673703121952713	bad_acc:0.7871893594679734
21:28:53 : 正确分类的样本数：14618，样本总数：16000，准确率：0.913625，ave_loss：0.00998687837272882599758291617035875
21:28:53 : ___train_one_bad_epoch___
21:29:16 : bad_epoch_loss:24.772423620335758	bad_acc:0.7651382569128456
21:30:16 : 正确分类的样本数：21891，样本总数：24000，准确率：0.912125，ave_loss：0.006555906031280756065166424028575425
21:30:16 : ___train_one_bad_epoch___
21:30:38 

22:26:29 : bad_epoch_loss:24.00541940657422	bad_acc:0.6275813790689534
22:27:29 : 正确分类的样本数：7474，样本总数：8000，准确率：0.93425，ave_loss：0.01535471342504024515391151420772076
22:27:29 : ___train_one_bad_epoch___
22:27:51 : bad_epoch_loss:25.32161503471434	bad_acc:0.6104305215260764
22:28:50 : 正确分类的样本数：14966，样本总数：16000，准确率：0.935375，ave_loss：0.00766513869166374276654385775327685
22:28:50 : ___train_one_bad_epoch___
22:29:13 : bad_epoch_loss:22.944695936283097	bad_acc:0.6212810640532027
22:30:12 : 正确分类的样本数：22403，样本总数：24000，准确率：0.9334583333333333，ave_loss：0.00519762048497796122612765
22:30:12 : ___train_one_bad_epoch___
22:30:35 : bad_epoch_loss:22.26221126737073	bad_acc:0.6191809590479525
22:31:34 : 正确分类的样本数：29870，样本总数：32000，准确率：0.9334375，ave_loss：0.00373371853493154057086259108036757
22:31:34 : ___train_one_bad_epoch___
22:31:57 : bad_epoch_loss:24.777453887742013	bad_acc:0.6065803290164509
22:32:56 : 正确分类的样本数：37373，样本总数：40000，准确率：0.934325，ave_loss：0.002943891799077391629332954436540604
22:32:56 :

23:02:01 : 正确分类的样本数：23419，样本总数：25520，准确率：0.9176724137931035，ave_loss：0.0153605500236153641166115
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      6380
           1       0.96      0.95      0.96      6380
           2       0.89      0.89      0.89      6380
           3       0.89      0.91      0.90      6380

    accuracy                           0.92     25520
   macro avg       0.92      0.92      0.92     25520
weighted avg       0.92      0.92      0.92     25520

23:02:01 : 准确率:0.9176724137931035
23:02:01 : store model
23:02:01 : __________________test end__________________
23:02:01 : train_acc:0.9425705329153605  train_epoch_loss:102.50138854980469  test_acc:0.9176724137931035  test_epoch_loss:392.001220703125
23:02:01 : ______________________________________________
23:02:01 : ______________________________________________
23:02:01 : _______________train epoch 7 end_______________
23:02:01 : _____________________________

In [18]:
cls2=fn_cls(device0)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
def train_from0(epoch_num):
    min_test_epoch_loss=999999
    for i in range(epoch_num):
        min_test_epoch_loss=train_one_epoch_from0(min_test_epoch_loss,device0,i,lmd=1)

    
# train_one_epoch(device0,0)
# train()

In [ ]:
from torch import optim
optimizer2 = optim.Adam(cls2.parameters(), lr=1e-6)

train_acc_l=[]
train_epoch_loss_l=[]
test_acc_l=[]
test_epoch_loss_l=[]
train_from0(10)

In [22]:
label_all,output_all=predict_loader(device0,test1_loader,cls)

准确率:0.8880485893416928
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      6380
           1       0.92      0.95      0.93      6380
           2       0.85      0.85      0.85      6380
           3       0.87      0.87      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [26]:
cls3=torch.load("../data/cls2_8_0.86577_317.2764.model",map_location=device0)
label_all,output_all=predict_loader(device0,test1_loader,cls3)

准确率:0.9185736677115988
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      6380
           1       0.95      0.97      0.96      6380
           2       0.89      0.88      0.89      6380
           3       0.89      0.92      0.90      6380

    accuracy                           0.92     25520
   macro avg       0.92      0.92      0.92     25520
weighted avg       0.92      0.92      0.92     25520



In [ ]:
label_all,output_all=predict_loader(device0,test1_loader,cls2)
#               precision    recall  f1-score   support

#            0       0.93      0.93      0.93      6380
#            1       0.96      0.96      0.96      6380
#            2       0.90      0.89      0.90      6380
#            3       0.90      0.92      0.91      6380

#     accuracy                           0.92     25520
#    macro avg       0.92      0.92      0.92     25520
# weighted avg       0.92      0.92      0.92     25520

# 23:27:24 : 准确率:0.9230015673981191

In [8]:
cls3=torch.load("../data/cls_bad_9_0.91708_396.36725.model",map_location=device0)
label_all,output_all=predict_loader(device0,test1_loader,cls3)

准确率:0.9170846394984326
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      6380
           1       0.94      0.97      0.95      6380
           2       0.90      0.88      0.89      6380
           3       0.90      0.91      0.90      6380

    accuracy                           0.92     25520
   macro avg       0.92      0.92      0.92     25520
weighted avg       0.92      0.92      0.92     25520



In [17]:
label_all,output_all=predict_loader(device0,test2_loader,cls)

准确率:0.8887931034482759
              precision    recall  f1-score   support

           0       0.92      0.88      0.90      6380
           1       0.92      0.95      0.93      6380
           2       0.86      0.84      0.85      6380
           3       0.86      0.88      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [16]:
cls3=torch.load("../data/cls2_8_0.86577_317.2764.model",map_location=device0)
label_all,output_all=predict_loader(device0,test2_loader,cls3)

准确率:0.8913009404388714
              precision    recall  f1-score   support

           0       0.93      0.88      0.90      6380
           1       0.92      0.95      0.94      6380
           2       0.86      0.84      0.85      6380
           3       0.85      0.88      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [14]:
label_all,output_all=predict_loader(device0,test2_loader,cls2)

准确率:0.8918495297805643
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      6380
           1       0.93      0.94      0.94      6380
           2       0.87      0.85      0.86      6380
           3       0.86      0.89      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [9]:
label_all,output_all=predict_loader(device0,test2_loader,cls3)

准确率:0.8906347962382445
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      6380
           1       0.92      0.95      0.94      6380
           2       0.87      0.84      0.85      6380
           3       0.86      0.88      0.87      6380

    accuracy                           0.89     25520
   macro avg       0.89      0.89      0.89     25520
weighted avg       0.89      0.89      0.89     25520



In [ ]:
from torch import optim
cls2=torch.load("../data/cls_6_0.88785_266.58456.model",map_location=device0)
optimizer2 = optim.Adam(cls2.parameters(), lr=1e-4)

for i in range(5):
    print("____________________epoch:"+str(i)+"____________________")
    epoch_loss=train_one_epoch(device0,0)
    print(epoch_loss)
    label_all,output_all=predict_loader(device0,test1_loader,cls2)


In [9]:
def predict(device,s_l,cls):
    with torch.no_grad():
        cls.to(device)
        cls.eval()
        text2id = tokenizer(
            s_l, max_length=100, padding='max_length', truncation=True, return_tensors="pt"
        )
        input_ids=text2id["input_ids"].to(device)
        mask=text2id["attention_mask"].to(device)
        output = cls(input_ids, attention_mask=mask)
        softmax = nn.Softmax(dim=1)
        output1=softmax(output)
        output2=output.argmax(dim=1)
        return output1,output2

In [10]:
s=['Echoes Repeats Success','"Stocks Finish Lower, Retail Sector Weighs"','Report indicates Wannstedt out','Conference Members Back Iraqi Efforts']
print(predict(device0,s,cls))
print(predict(device0,s,cls2))

(tensor([[1.6244e-02, 9.7378e-01, 5.3007e-03, 4.6791e-03],
        [6.1406e-02, 1.4047e-03, 9.3121e-01, 5.9830e-03],
        [7.6313e-03, 9.8954e-01, 2.2822e-03, 5.4985e-04],
        [9.9728e-01, 8.1377e-04, 1.0767e-03, 8.2985e-04]], device='cuda:6'), tensor([1, 2, 1, 0], device='cuda:6'))
(tensor([[1.6244e-02, 9.7378e-01, 5.3007e-03, 4.6791e-03],
        [6.1406e-02, 1.4047e-03, 9.3121e-01, 5.9830e-03],
        [7.6313e-03, 9.8954e-01, 2.2822e-03, 5.4985e-04],
        [9.9728e-01, 8.1377e-04, 1.0767e-03, 8.2985e-04]], device='cuda:6'), tensor([1, 2, 1, 0], device='cuda:6'))


In [38]:
import time
end=time.time()
torch.save(cls2,"../data/cls2_bad_"+str(end)+".model")

In [ ]:
# 0.02694982503663435
# 0.20794415416798367
